In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= BeautifulSoup(source,'lxml')
# print(soup.prettify())

Use Beatifulsoup liberary to get table from Wiki website

In [5]:
soup = BeautifulSoup(source,'lxml')
table = soup.find('div', class_="mw-parser-output").table
table_rows = table.find_all('tr')
l=[]
for tr in table_rows:
    td = tr.find_all('td')
#     print(td)
    row = [tr.text for tr in td]
#     print(row)
    l.append(row)
table_=pd.DataFrame(l, columns=["Postcode", "Borough","Neighborhood"])
df=table_
df.head(5)

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


Start edit the table to get the target assignment

In [6]:
df_=df[df['Borough']!='Not assigned']
df_.drop(df_.index[0],inplace= True)
df_.reset_index(drop=True,inplace=True)
df_ = df_.replace('\n',',', regex=True)
df_['Neighborhood'] = np.where(df_['Neighborhood'] == "Not assigned,", df_['Borough']+",", df_['Neighborhood'])
df_.head(5)

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M5A,Downtown Toronto,"Regent Park,"
4,M6A,North York,"Lawrence Heights,"


In [7]:
def insert_comma(x):
    y=x[:-1]
    return y   

Final step to get Datafarame and shape target

In [8]:
A=df_.groupby('Postcode')['Borough'].first()
B=df_.groupby('Postcode')['Neighborhood'].sum().apply(lambda x: insert_comma(x))
dataframe=pd.DataFrame([A,B]).transpose().reset_index()
dataframe.head(5)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
dataframe.shape

(103, 3)

In [10]:
# dataframe['Postcode'].head(5)

In [11]:
# ! pip install geocoder

In [12]:
# import geocoder # import geocoder

In [13]:
# dataframe['latitude']=None
# dataframe['longitude']=None

# for postal_code in dataframe['Postcode']:
# # initialize your variable to None
#     lat_lng_coords = None
# # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         print(postal_code)
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#         print('NA')
    
#     print(postal_code)

#     dataframe['latitude'].append(lat_lng_coords[0])
#     dataframe['latitude'].append(lat_lng_coords[1])
#doesnt work

Start read CSV file to get the Lat and Long for Neighborhood

In [14]:
df = pd.read_csv (r'C:\Users\HP\Desktop\projects\Coursera_Capstone\Geospatial_Coordinates.csv')
df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# dataframe

Merge the two Dataframe to get the required target

In [16]:
data_frame=pd.merge(df, dataframe, left_on='Postal Code', right_on='Postcode')
data_frame=data_frame[['Postcode','Borough','Neighborhood','Latitude','Longitude']]
data_frame.head(5)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Start the steps of clustering Toronto neighborhood as Newyork example

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_frame['Latitude'], data_frame['Longitude'], data_frame['Borough'], data_frame['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [20]:
CLIENT_ID = 'SYNKAEL54LI1HUIAAZJ5VCZT3OISLVZSEAAASQDTUWRCXAVN' # your Foursquare ID
CLIENT_SECRET = 'JSJZWCJ3LNYT0YLLWXZKXXBFZ3JMJGYPX420H2AMWAQJF55K' # your Foursquare Secret
VERSION = '20140715' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SYNKAEL54LI1HUIAAZJ5VCZT3OISLVZSEAAASQDTUWRCXAVN
CLIENT_SECRET:JSJZWCJ3LNYT0YLLWXZKXXBFZ3JMJGYPX420H2AMWAQJF55K


In [23]:
LIMIT=200
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
#         print(venues_list)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
 
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
#     print(nearby_venues.head(2))
    
    return(nearby_venues)

In [24]:
Toronto_venues = getNearbyVenues(names=data_frame['Neighborhood'],latitudes=data_frame['Latitude'],longitudes=data_frame['Longitude'])

In [25]:
Toronto_venues.shape

(4859, 7)

In [26]:
Toronto_venues.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge,Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
2,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
4,"Rouge,Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant


In [27]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")


# # add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 




In [33]:
Toronto_onehot.head(5)
Toronto_onehot.columns.values
Toronto_onehot = Toronto_onehot[['Neighborhood' ,'Accessories Store', 'Afghan Restaurant', 'African Restaurant',
       'Airport', 'Airport Lounge', 'American Restaurant', 'Amphitheater',
       'Animal Shelter', 'Antique Shop', 'Aquarium', 'Art Gallery',
       'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant',
       'Athletics & Sports', 'Auto Dealership', 'Auto Garage',
       'Automotive Shop', 'BBQ Joint', 'Baby Store', 'Badminton Court',
       'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Field',
       'Baseball Stadium', 'Basketball Stadium', 'Beach', 'Beach Bar',
       'Beer Bar', 'Beer Store', 'Belgian Restaurant', 'Bike Shop',
       'Bistro', 'Bookstore', 'Boutique', 'Bowling Alley',
       'Brazilian Restaurant', 'Breakfast Spot', 'Brewery', 'Bridal Shop',
       'Bridge', 'Bubble Tea Shop', 'Buffet', 'Building', 'Burger Joint',
       'Burrito Place', 'Bus Line', 'Bus Station', 'Bus Stop',
       'Business Service', 'Butcher', 'Cafeteria', 'Café',
       'Cajun / Creole Restaurant', 'Camera Store', 'Candy Store',
       'Cantonese Restaurant', 'Caribbean Restaurant', 'Castle',
       'Cemetery', 'Cheese Shop', 'Chinese Restaurant', 'Chiropractor',
       'Chocolate Shop', 'Church', 'Churrascaria', 'Climbing Gym',
       'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Gym',
       'College Lab', 'College Quad', 'College Rec Center',
       'College Stadium', 'College Theater', 'Comedy Club',
       'Comfort Food Restaurant', 'Comic Shop', 'Community Center',
       'Concert Hall', 'Construction & Landscaping', 'Convenience Store',
       'Cosmetics Shop', 'Coworking Space', 'Creperie',
       'Cuban Restaurant', 'Cupcake Shop', 'Curling Ice', 'Dance Studio',
       'Deli / Bodega', 'Department Store', 'Design Studio',
       'Dessert Shop', 'Dim Sum Restaurant', 'Diner', 'Discount Store',
       'Dive Bar', 'Dog Run', 'Doner Restaurant', 'Donut Shop',
       'Dumpling Restaurant', 'Eastern European Restaurant',
       'Electronics Store', 'Empanada Restaurant', 'Ethiopian Restaurant',
       'Event Space', 'Falafel Restaurant', 'Farm', 'Farmers Market',
       'Fast Food Restaurant', 'Field', 'Filipino Restaurant',
       'Fireworks Store', 'Fish & Chips Shop', 'Fish Market',
       'Flea Market', 'Flower Shop', 'Food', 'Food & Drink Shop',
       'Food Court', 'Food Truck', 'Fountain', 'French Restaurant',
       'Fried Chicken Joint', 'Frozen Yogurt Shop',
       'Fruit & Vegetable Store', 'Furniture / Home Store', 'Gaming Cafe',
       'Garden', 'Garden Center', 'Gas Station', 'Gastropub', 'Gay Bar',
       'General Entertainment', 'German Restaurant', 'Gift Shop',
       'Golf Course', 'Gourmet Shop', 'Greek Restaurant', 'Grocery Store',
       'Gym', 'Gym / Fitness Center', 'Gym Pool', 'Hakka Restaurant',
       'Halal Restaurant', 'Harbor / Marina', 'Hardware Store',
       'Hawaiian Restaurant', 'Health & Beauty Service',
       'Health Food Store', 'Historic Site', 'History Museum',
       'Hobby Shop', 'Hockey Arena', 'Home Service', 'Hookah Bar',
       'Hostel', 'Hot Dog Joint', 'Hotel', 'Hotel Bar',
       'Hotpot Restaurant', 'Housing Development', 'IT Services',
       'Ice Cream Shop', 'Indian Chinese Restaurant', 'Indian Restaurant',
       'Indie Movie Theater', 'Indie Theater', 'Indonesian Restaurant',
       'Intersection', 'Italian Restaurant', 'Japanese Restaurant',
       'Jazz Club', 'Jewelry Store', 'Jewish Restaurant', 'Juice Bar',
       'Karaoke Bar', 'Kids Store', 'Kitchen Supply Store',
       'Korean Restaurant', 'Lake', 'Latin American Restaurant',
       'Laundry Service', 'Light Rail Station', 'Lingerie Store',
       'Liquor Store', 'Lounge', 'Mac & Cheese Joint', 'Malay Restaurant',
       'Market', 'Martial Arts Dojo', 'Massage Studio', 'Mattress Store',
       'Mediterranean Restaurant', "Men's Store", 'Metro Station',
       'Mexican Restaurant', 'Middle Eastern Restaurant',
       'Miscellaneous Shop', 'Mobile Phone Shop',
       'Modern European Restaurant', 'Monument / Landmark',
       'Moroccan Restaurant', 'Motorcycle Shop', 'Movie Theater',
       'Museum', 'Music School', 'Music Store', 'Music Venue',
       'Nail Salon','New American Restaurant',
       'Nightclub', 'Noodle House', 'Office', 'Optical Shop',
       'Organic Grocery', 'Other Great Outdoors', 'Other Repair Shop',
       'Outdoors & Recreation', 'Pakistani Restaurant',
       'Paper / Office Supplies Store', 'Park', 'Pastry Shop',
       'Performing Arts Venue', 'Persian Restaurant', 'Pet Store',
       'Pharmacy', 'Photography Lab', 'Pide Place', 'Pie Shop',
       'Pilates Studio', 'Pizza Place', 'Playground', 'Plaza',
       'Poke Place', 'Polish Restaurant', 'Pool', 'Pool Hall',
       'Portuguese Restaurant', 'Poutine Place', 'Print Shop', 'Pub',
       'Ramen Restaurant', 'Record Shop', 'Recreation Center',
       'Rental Car Location', 'Residential Building (Apartment / Condo)',
       'Restaurant', 'River', 'Road', 'Rock Climbing Spot', 'Rock Club',
       'Roof Deck', 'Sake Bar', 'Salad Place', 'Salon / Barbershop',
       'Sandwich Place', 'Scenic Lookout', 'School', 'Sculpture Garden',
       'Seafood Restaurant', 'Shanghai Restaurant', 'Shoe Store',
       'Shop & Service', 'Shopping Mall', 'Skate Park', 'Skating Rink',
       'Ski Area', 'Ski Chalet', 'Smoke Shop', 'Smoothie Shop',
       'Snack Place', 'Soccer Field', 'Soccer Stadium', 'Social Club',
       'Soup Place', 'South American Restaurant',
       'Southern / Soul Food Restaurant', 'Souvlaki Shop', 'Spa',
       'Spanish Restaurant', 'Speakeasy', 'Sporting Goods Shop',
       'Sports Bar', 'Sports Club', 'Sri Lankan Restaurant',
       'Stationery Store', 'Steakhouse', 'Storage Facility', 'Street Art',
       'Supermarket', 'Supplement Shop', 'Sushi Restaurant', 'Taco Place',
       'Tailor Shop', 'Taiwanese Restaurant', 'Tanning Salon',
       'Tapas Restaurant', 'Tea Room', 'Tech Startup', 'Tennis Court',
       'Thai Restaurant', 'Theater', 'Theme Restaurant',
       'Thrift / Vintage Store', 'Tibetan Restaurant', 'Toy / Game Store',
       'Track', 'Trail', 'Train Station', 'Tree', 'Turkish Restaurant',
       'Udon Restaurant', 'University', 'Vegetarian / Vegan Restaurant',
       'Video Game Store', 'Video Store', 'Vietnamese Restaurant',
       'Warehouse Store', 'Waste Facility', 'Whisky Bar', 'Wine Bar',
       'Wine Shop', 'Wings Joint', "Women's Store", 'Yoga Studio', 'Zoo']]
# Toronto_onehot.head(5)

In [34]:
Toronto_onehot.shape

(4859, 331)

In [35]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head(2)

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South Am

In [ ]:
# num_top_venues = 5

# for hood in Toronto_grouped['Neighborhood']:
# #     print("----"+hood+"----")
#     temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Hotel,Theater,American Restaurant,Sushi Restaurant,Asian Restaurant,Concert Hall,Bakery,Cosmetics Shop
1,Agincourt,Chinese Restaurant,Shopping Mall,Supermarket,Coffee Shop,Bakery,Sandwich Place,Caribbean Restaurant,Pizza Place,Sri Lankan Restaurant,Lounge
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Bakery,Noodle House,Pizza Place,Park,Coffee Shop,Fast Food Restaurant,Event Space,Malay Restaurant,Caribbean Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Coffee Shop,Beer Store,Park,Bus Line,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Hardware Store
4,"Alderwood,Long Branch",Discount Store,Pharmacy,Park,Convenience Store,Pizza Place,Gym,Skating Rink,Donut Shop,Shopping Mall,Liquor Store


In [51]:
kclusters = 8

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 6, 6, 4, 6, 7, 6, 7, 0, 0])

In [53]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels',axis=1,inplace= True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = data_frame

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!
Toronto_merged_cluster=Toronto_merged.groupby('Cluster Labels')


In [54]:
Toronto_merged.dropna(inplace=True)
Toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,7.0,Fast Food Restaurant,Park,Arts & Crafts Store,Filipino Restaurant,Fruit & Vegetable Store,Martial Arts Dojo,Caribbean Restaurant,Chinese Restaurant,Paper / Office Supplies Store,Greek Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,5.0,Playground,Burger Joint,Italian Restaurant,Park,Breakfast Spot,Flea Market,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,4.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Pharmacy,Food & Drink Shop,Grocery Store,Filipino Restaurant,Sports Bar,Smoothie Shop,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.0,Park,Coffee Shop,Fast Food Restaurant,Pharmacy,Electronics Store,Chinese Restaurant,Indian Restaurant,Fireworks Store,Filipino Restaurant,Field
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,7.0,Bakery,Coffee Shop,Indian Restaurant,Thai Restaurant,Athletics & Sports,Sporting Goods Shop,Fried Chicken Joint,Music Store,Fast Food Restaurant,Caribbean Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4.0,Fast Food Restaurant,Coffee Shop,Convenience Store,Grocery Store,Restaurant,Bowling Alley,Pizza Place,Japanese Restaurant,Sandwich Place,Train Station
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,4.0,Coffee Shop,Discount Store,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Pizza Place,Convenience Store,Department Store,Metro Station,Light Rail Station
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,7.0,Coffee Shop,Intersection,Fast Food Restaurant,Bakery,Sandwich Place,Grocery Store,Diner,Park,Bus Station,Pub
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,4.0,Pizza Place,Beach,Fast Food Restaurant,Sports Bar,Park,Cajun / Creole Restaurant,Furniture / Home Store,Burger Joint,Fireworks Store,Filipino Restaurant
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,Café,Fast Food Restaurant,Thai Restaurant,Bank,Park,Skating Rink,Gym,Gym Pool,Diner,General Entertainment


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examin Clusters

In [56]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,0.0,Café,Fast Food Restaurant,Thai Restaurant,Bank,Park,Skating Rink,Gym,Gym Pool,Diner,General Entertainment
21,North York,0.0,Café,Korean Restaurant,Middle Eastern Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Fried Chicken Joint,Massage Studio,Shopping Mall,Electronics Store
40,East York,0.0,Coffee Shop,Café,Greek Restaurant,Pizza Place,Park,Ethiopian Restaurant,Portuguese Restaurant,Breakfast Spot,Bar,Gastropub
41,East Toronto,0.0,Greek Restaurant,Coffee Shop,Café,Pub,Italian Restaurant,Pizza Place,Portuguese Restaurant,Bakery,Sandwich Place,Fast Food Restaurant
43,East Toronto,0.0,Coffee Shop,Bakery,American Restaurant,Bar,Café,Brewery,Italian Restaurant,Vietnamese Restaurant,French Restaurant,Diner
44,Central Toronto,0.0,College Gym,Café,Park,Bus Line,Gym / Fitness Center,Bookstore,Trail,College Quad,Coffee Shop,Empanada Restaurant
46,Central Toronto,0.0,Italian Restaurant,Coffee Shop,Sporting Goods Shop,Mexican Restaurant,Skating Rink,Café,Park,Diner,Sushi Restaurant,Bakery
51,Downtown Toronto,0.0,Park,Japanese Restaurant,Gastropub,Café,Diner,Coffee Shop,Caribbean Restaurant,Garden,Beer Store,Taiwanese Restaurant
55,Downtown Toronto,0.0,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Breakfast Spot,Cosmetics Shop,Seafood Restaurant,American Restaurant
56,Downtown Toronto,0.0,Coffee Shop,Hotel,Café,Beer Bar,Restaurant,Japanese Restaurant,Bakery,Park,Gym,Seafood Restaurant


In [57]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,1.0,Vietnamese Restaurant,Baseball Field,Restaurant,Zoo,Fast Food Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [58]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Etobicoke,2.0,Coffee Shop,Dog Run,Rental Car Location,Lounge,Zoo,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [45]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,3.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Pharmacy,Food & Drink Shop,Grocery Store,Filipino Restaurant,Sports Bar,Smoothie Shop,Fried Chicken Joint
5,Scarborough,3.0,Fast Food Restaurant,Coffee Shop,Convenience Store,Grocery Store,Restaurant,Bowling Alley,Pizza Place,Japanese Restaurant,Sandwich Place,Train Station
6,Scarborough,3.0,Coffee Shop,Discount Store,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Pizza Place,Convenience Store,Department Store,Metro Station,Light Rail Station
8,Scarborough,3.0,Pizza Place,Beach,Fast Food Restaurant,Sports Bar,Park,Cajun / Creole Restaurant,Furniture / Home Store,Burger Joint,Fireworks Store,Filipino Restaurant
29,North York,3.0,Coffee Shop,Pizza Place,Restaurant,Furniture / Home Store,Japanese Restaurant,Bar,Bank,Caribbean Restaurant,Sandwich Place,Massage Studio
33,North York,3.0,Hotel,Fast Food Restaurant,Pizza Place,Discount Store,Grocery Store,Pharmacy,Coffee Shop,Kitchen Supply Store,Athletics & Sports,Chinese Restaurant
35,East York,3.0,Brewery,Fast Food Restaurant,Pizza Place,Bank,Gastropub,Bakery,Gym / Fitness Center,Coffee Shop,Pharmacy,Athletics & Sports
72,North York,3.0,Fast Food Restaurant,Grocery Store,Coffee Shop,Gym,Pub,Photography Lab,Latin American Restaurant,Park,Bookstore,Sandwich Place
98,York,3.0,Coffee Shop,Train Station,Pizza Place,Soccer Field,Discount Store,Diner,Sandwich Place,Furniture / Home Store,Pharmacy,Fried Chicken Joint
101,Etobicoke,3.0,Grocery Store,Pizza Place,Coffee Shop,Beer Store,Park,Bus Line,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Hardware Store


In [59]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,4.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Pharmacy,Food & Drink Shop,Grocery Store,Filipino Restaurant,Sports Bar,Smoothie Shop,Fried Chicken Joint
5,Scarborough,4.0,Fast Food Restaurant,Coffee Shop,Convenience Store,Grocery Store,Restaurant,Bowling Alley,Pizza Place,Japanese Restaurant,Sandwich Place,Train Station
6,Scarborough,4.0,Coffee Shop,Discount Store,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Pizza Place,Convenience Store,Department Store,Metro Station,Light Rail Station
8,Scarborough,4.0,Pizza Place,Beach,Fast Food Restaurant,Sports Bar,Park,Cajun / Creole Restaurant,Furniture / Home Store,Burger Joint,Fireworks Store,Filipino Restaurant
13,Scarborough,4.0,Pizza Place,Pharmacy,Coffee Shop,Sandwich Place,Fast Food Restaurant,Shopping Mall,Bank,Taiwanese Restaurant,Fried Chicken Joint,Market
15,Scarborough,4.0,Chinese Restaurant,Coffee Shop,Bakery,Fast Food Restaurant,Pharmacy,Other Great Outdoors,Tennis Court,Sandwich Place,Discount Store,Grocery Store
29,North York,4.0,Coffee Shop,Pizza Place,Restaurant,Furniture / Home Store,Japanese Restaurant,Bar,Bank,Caribbean Restaurant,Sandwich Place,Massage Studio
33,North York,4.0,Hotel,Fast Food Restaurant,Pizza Place,Discount Store,Grocery Store,Pharmacy,Coffee Shop,Kitchen Supply Store,Athletics & Sports,Chinese Restaurant
35,East York,4.0,Brewery,Fast Food Restaurant,Pizza Place,Bank,Gastropub,Bakery,Gym / Fitness Center,Coffee Shop,Pharmacy,Athletics & Sports
72,North York,4.0,Fast Food Restaurant,Grocery Store,Coffee Shop,Gym,Pub,Photography Lab,Latin American Restaurant,Park,Bookstore,Sandwich Place


In [60]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,5.0,Playground,Burger Joint,Italian Restaurant,Park,Breakfast Spot,Flea Market,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant
3,Scarborough,5.0,Park,Coffee Shop,Fast Food Restaurant,Pharmacy,Electronics Store,Chinese Restaurant,Indian Restaurant,Fireworks Store,Filipino Restaurant,Field
23,North York,5.0,Park,Restaurant,Coffee Shop,Convenience Store,Intersection,French Restaurant,Optical Shop,Tennis Court,Bank,Grocery Store
31,North York,5.0,Park,Coffee Shop,Vietnamese Restaurant,Shopping Mall,Pizza Place,Bank,Spa,Filipino Restaurant,Field,Fish Market
91,Etobicoke,5.0,Park,Italian Restaurant,Ice Cream Shop,Eastern European Restaurant,Shopping Mall,Fish Market,Falafel Restaurant,Dumpling Restaurant,Flower Shop,Electronics Store
96,North York,5.0,Electronics Store,Bank,Bakery,Shopping Mall,Park,Empanada Restaurant,Pharmacy,Pizza Place,Italian Restaurant,Event Space


In [61]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,6.0,Chinese Restaurant,Shopping Mall,Supermarket,Coffee Shop,Bakery,Sandwich Place,Caribbean Restaurant,Pizza Place,Sri Lankan Restaurant,Lounge
14,Scarborough,6.0,Chinese Restaurant,Bakery,Noodle House,Pizza Place,Park,Coffee Shop,Fast Food Restaurant,Event Space,Malay Restaurant,Caribbean Restaurant
17,North York,6.0,Park,Pharmacy,Coffee Shop,Chinese Restaurant,Bank,Korean Restaurant,Convenience Store,Fast Food Restaurant,Shopping Mall,Housing Development
19,North York,6.0,Bank,Japanese Restaurant,Chinese Restaurant,Grocery Store,Park,Fast Food Restaurant,Trail,Shopping Mall,Skate Park,Café
24,North York,6.0,Pharmacy,Park,Pizza Place,Bus Line,Butcher,Eastern European Restaurant,Bakery,Discount Store,Coffee Shop,Convenience Store
25,North York,6.0,Park,Bus Stop,Pharmacy,Convenience Store,Shopping Mall,Food & Drink Shop,Café,Supermarket,Skating Rink,Train Station
74,York,6.0,Pharmacy,Bus Stop,Mexican Restaurant,Park,Cosmetics Shop,Bus Line,Fast Food Restaurant,Market,Falafel Restaurant,Beer Store
89,Etobicoke,6.0,Discount Store,Pharmacy,Park,Convenience Store,Pizza Place,Gym,Skating Rink,Donut Shop,Shopping Mall,Liquor Store
93,Etobicoke,6.0,Pharmacy,Convenience Store,Café,Skating Rink,Shopping Mall,Bank,Bakery,Park,Grocery Store,Golf Course
94,Etobicoke,6.0,Hotel,Pizza Place,Convenience Store,Bank,Park,American Restaurant,Theater,Fish & Chips Shop,Mexican Restaurant,Grocery Store


In [62]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 7, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,7.0,Fast Food Restaurant,Park,Arts & Crafts Store,Filipino Restaurant,Fruit & Vegetable Store,Martial Arts Dojo,Caribbean Restaurant,Chinese Restaurant,Paper / Office Supplies Store,Greek Restaurant
4,Scarborough,7.0,Bakery,Coffee Shop,Indian Restaurant,Thai Restaurant,Athletics & Sports,Sporting Goods Shop,Fried Chicken Joint,Music Store,Fast Food Restaurant,Caribbean Restaurant
7,Scarborough,7.0,Coffee Shop,Intersection,Fast Food Restaurant,Bakery,Sandwich Place,Grocery Store,Diner,Park,Bus Station,Pub
10,Scarborough,7.0,Electronics Store,Coffee Shop,Furniture / Home Store,Asian Restaurant,Bakery,Pharmacy,Indian Restaurant,Burger Joint,Automotive Shop,Fast Food Restaurant
11,Scarborough,7.0,Middle Eastern Restaurant,Grocery Store,Intersection,Pizza Place,Burger Joint,Supermarket,Badminton Court,Korean Restaurant,Gas Station,Seafood Restaurant
18,North York,7.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Sandwich Place,Bakery,Pizza Place,Electronics Store,Juice Bar,Smoothie Shop
22,North York,7.0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Japanese Restaurant,Fast Food Restaurant,Pizza Place,Korean Restaurant,Sandwich Place,Sushi Restaurant,Café
26,North York,7.0,Coffee Shop,Japanese Restaurant,Burger Joint,Pizza Place,Spa,Salad Place,Greek Restaurant,Thai Restaurant,Bank,Mobile Phone Shop
27,North York,7.0,Restaurant,Gym,Coffee Shop,American Restaurant,Supermarket,Beer Store,Asian Restaurant,Japanese Restaurant,Concert Hall,Middle Eastern Restaurant
28,North York,7.0,Coffee Shop,Bridal Shop,Dog Run,Fried Chicken Joint,Sushi Restaurant,Supermarket,Mediterranean Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Ski Chalet
